In [10]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import random, pickle, math, warnings
import transformers, torch
import gc, os, logging
from math import exp
from typing import List, Optional, Union
warnings.simplefilter('ignore')


In [11]:
# adjust display options
pd.set_option('display.max_colwidth', None)

# load submission file = input data
df = pd.read_csv('../input/santa-2024/sample_submission.csv')
df

,id,text
0,0,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge
1,1,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and
2,2,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice
3,3,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap
4,4,hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle
5,5,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle


In [12]:
tokens = df.text.values
tokens = ' '.join(tokens)
tokens = tokens.split(' ')
tokens = dict(Counter(tokens))
print(len(tokens))
#print(tokens)
print('-'*20)
print(' '.join(k for k in tokens))

89
--------------------
advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh workshop stocking holly jingle beard naughty nice sing of is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy wonder believe dream hope peace joy merry season greeting card wrapping paper bow cookie milk star wish wreath angel to in that have it not with as you from we kaggle


In [13]:
MODEL_PATH = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"

class ParticipantVisibleError(Exception):
    print(Exception)
    pass

class PerplexityCalculator:
    def __init__(self,):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_PATH)
        self.model = transformers.AutoModelForCausalLM.from_pretrained(MODEL_PATH, device_map="cpu", torch_dtype=torch.float32,)
        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        self.model.eval()
        #self.model.to(DEVICE)

    def get_perplexity(self, input_texts: Union[str, List[str]],) -> Union[float, List[float]]:
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts
        loss_list = []
        with torch.no_grad():
            for text in input_texts:
                text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"
                minputs = self.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False,)
                #if 'token_type_ids' in minputs: minputs.pop('token_type_ids')
                #minputs = {k: v.to(DEVICE) for k, v in minputs.items()}
                output = self.model(**minputs, use_cache=False)
                logits = output['logits']
                slogits = logits[..., :-1, :].contiguous()
                slabels = minputs['input_ids'][..., 1:].contiguous()
                loss = self.loss_fct(slogits.view(-1, slogits.size(-1)), slabels.view(-1))
                sequence_loss = loss.sum() / len(loss)
                loss_list.append(sequence_loss.cpu().item())
        ppl = [exp(i) for i in loss_list]
        return ppl[0] if single_input else ppl

scorer = PerplexityCalculator()

<class 'Exception'>


Loading checkpoint shards: 100%|██████████| 8/8 [00:01<00:00,  4.08it/s]


In [14]:
tokens = "advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh workshop stocking holly jingle beard naughty nice sing of is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy wonder believe dream hope peace joy merry season greeting card wrapping paper bow cookie milk star wish wreath angel to in that have it not with as you from we kaggle"
text_with_special = f"{scorer.tokenizer.bos_token}{tokens}{scorer.tokenizer.eos_token}"
minputs = scorer.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False,)
minputs['input_ids']

tensor([[     2, 104828,  67905,  52931,   2730,  43485, 136507,   7727, 165493,
          29138, 103360,   1513,  80108,    541,   5376,   2734,   9902,   6109,
          44528,    573,   6284,   3354,  10084,    578,    597,   1731,  23675,
          42768,  17196,  22867,  12083, 138763, 198447,  16621,  99946,  16573,
           2660,  14111, 155702,  20257,  77515, 108548, 204063,  38175,  97840,
           4866,   2800,    576,    603,   7812,   3532,  10228,    748,  14660,
           1965, 215898,  28162,  83096,    881,   9437,   8529, 112671, 149218,
           7815,   1312,    869,   9471,  23144,  13171,  25720,  24754,   2398,
           7474,  12849,   5144,   4564,   6523,   4077,   7124,  10300,  46301,
           3891,  32338,   4076,  56178,   4368,   7181,  17467,   9512,   2343,
           6199,  58409,  22448,    577,    575,    674,    791,    665,    780,
            675,    685,    692,    774,    783, 124555,   2315,      1]])

In [17]:
past = {}

t = """reindeer mistletoe elf scrooge gingerbread chimney fireplace ornament family advent
reindeer mistletoe elf scrooge gingerbread ornament advent family fireplace chimney sleep drive walk jump laugh give and bake the night
magi yuletide cheer grinch carol holly jingle naughty nice polar beard sleigh workshop chimney stocking nutcracker holiday ornament decorations gifts
sleigh of the magi yuletide cheer is unwrap gifts and decorations ornament holly stocking chimney carol grinch naughty nice polar beard nutcracker visit workshop eat relax sing holiday cheer jingle
wreath it have merry season hohoho and to you from the star of wonder workshop that night not with joy as we believe in hope peace milk cookie eggnog fruitcake chocolate candy peppermint candle snowglobe angel poinsettia wrapping paper bow greeting card wish dream fireplace kaggle toy doll game puzzle
eggnog beard mistletoe yuletide fruitcake scrooge nutcracker holly wreath gingerbread cookie snowglobe reindeer magi angel star decorations merry jingle and the season of joy and wonder and peace to you from the family of the grinch with that holiday cheer is not as cheer unwrap gifts laugh jump hohoho sing carol in sleigh drive visit chimney chimney elf naughty nice eat bake sleep dream relax chocolate milk peppermint candy ornament poinsettia stocking fireplace fireplace advent candle wish hope believe give greeting card ornament wrapping paper toy doll bow game night puzzle polar night walk it kaggle workshop we have workshop"""

df['text'] = t.split('\n')
df['score'] = df['text'].map(lambda x: scorer.get_perplexity(x))
df[['id','text']].to_csv("submission.csv", index=False)
df['score']

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


0    495.686221
1    503.860566
2    320.924809
3    281.041973
4    166.492962
5     91.406795
Name: score, dtype: float64

In [18]:
%%time

def get_best_plex(df, past):
    l = 2
    df['score'] = df['text'].map(lambda x: scorer.get_perplexity(x))
    print(np.mean(df['score'].values))
    for r in range(len(df)):
        #get best past
        print("row: ", r, df.at[r, 'score'])
        t = df['text'][r].split(' ')
        for k in past:
            kp = k.split(' ')
            if len(kp)==len(t):
                if sorted(kp) == sorted(t):
                    if df['score'][r] > past[k]:
                        df.at[r, 'score'] = past[k]
                        df.at[r, 'text'] = k
        print("row: ", r, df.at[r, 'score'])
        #for i in tqdm(range(l)):
        for i in range(l):
            t = df['text'][r].split(' ')
            random.shuffle(t)
            t = ' '.join(t)
            if t not in past:
                s = scorer.get_perplexity(t)
                past[t]=s
                if s < df['score'][r]:
                    df.at[r, 'score'] = s
                    df.at[r, 'text'] = t
                    print(r, i, "New Score: ", s, np.mean(df['score'].values))
        with open('past0.pickle', 'wb') as f:
            pickle.dump(past, f, protocol=pickle.HIGHEST_PROTOCOL)
    print('MEAN SCORE: ', np.mean(df['score'].values))
    return df[['id','text']], past

df, past = get_best_plex(df, past)
# df[['id','text']].to_csv("submission.csv", index=False)

309.90222121987705
row:  0 495.6862210096499
row:  0 495.6862210096499
row:  1 503.86056638907536
row:  1 503.86056638907536
row:  2 320.92480898643896
row:  2 320.92480898643896
row:  3 281.0419730096703
row:  3 281.0419730096703
row:  4 166.49296246948435
row:  4 166.49296246948435
row:  5 91.4067954549437
row:  5 91.4067954549437
MEAN SCORE:  309.90222121987705
CPU times: user 17min 30s, sys: 40.6 s, total: 18min 10s
Wall time: 27.3 s
